# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute_target import ComputeTargetException
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.widgets import RunDetailss

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'forest-fires-exp'

experiment=Experiment(ws, experiment_name)

In [ ]:
ds = TabularDatasetFactory.from_delimited_files("http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv")

In [ ]:
aml_compute_target = "cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=aml_compute_target)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)

    aml_compute.wait_for_completion(show_output=True)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=aml_compute,
    task='regression',
    primary_metric='normalised_mean_absolute_error',
    training_data = ds,
    label_column_name = 'area',
    n_cross_validations=5,
    **automl_settings
)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = automl_exp.get_output()
print(best_run)
print(fitted_model)
# best_run.get_file_names()

In [ ]:
#TODO: Save the best model
automl_model = best_run.register_model(model_name = 'best-model-automl',model_path='outputs/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
env = Environment.get(ws, "AzureML-Tutorial").clone("Inference_Env")

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)


deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, enable_app_insights=True)
service = Model.deploy(ws, "aciservice", [automl_model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [1]:
import requests
import json

scoring_uri = service.scoring_uri

headers = {'Content-Type': 'application/json'}

data = {"data":
        [
          {
            "X":8,
            "Y":6,
            "month":"dec",
            "day":"wed",
            "FFMC":84,
            "DMC":27.8,
            "DC":354.6,
            "ISI":5.3,
            "temp":5.1,
            "RH":61,
            "wind":8,
            "rain":0
          },
          {
            "X":4,
            "Y":6,
            "month":"dec",
            "day":"thu",
            "FFMC":84.6,
            "DMC":26.4,
            "DC":352,
            "ISI":2,
            "temp":5.1,
            "RH":61,
            "wind":4.9,
            "rain":0
            
          }
      ]
    }

# Convert to JSON string
input_data = json.dumps(data)

resp = requests.post(scoring_uri, input_data, headers=headers)

print("Response Status Code : ", resp.status_code)
print("Predicted Burnt Area : ",resp.text)

SyntaxError: invalid syntax (<ipython-input-1-c5da8030b294>, line 12)

In [ ]:
print(service.get_logs())

In [ ]:
service.delete()